In [ ]:
CSV = "data/gesamte_akte.csv"
file = open(CSV, "r", encoding="utf-8").read()

In [ ]:
lines = file.splitlines()
raw_matrix = [[(c if (c:=cell.strip()) != "" else None) for cell in line.split(";")]
              for line in file.splitlines()]

In [ ]:
# Check if all page-headers are followed by an empty line => Sieht danach aus
for i, row in enumerate(raw_matrix):
    if all(c is None for c in row):
        # print(raw_matrix[i-1] if i > 0 else None)
        pass

In [ ]:
# gibt alle Labor-Blöcke aus
labs_sections = []
for i, row in enumerate(raw_matrix):
    if (
        row and len(row) > 0
        and isinstance(row[0], str)
        and row[0].startswith("Labor:")
        and (i + 1) < len(raw_matrix)
        and raw_matrix[i+1] is not None
        and len(raw_matrix[i+1]) > 20
    ):
        labs_sections.append(row[0])

labs_sections = list(dict.fromkeys(labs_sections))
# labs_sections

In [ ]:
# Erste 5 Zeilen NACH jeder Labor-Section ausgeben
for labs_section in labs_sections:
    # print(labs_section)
    for i, row in enumerate(raw_matrix):
        if row and row[0] == labs_section:
            for nxt in raw_matrix[i+1:i+6]:
                # print(len(nxt))
                pass

In [ ]:
# bereinigt die csv von den ganzen page-headers
headers = []
skip = set()
skip.add(len(lines)-1)
for i, line in enumerate(lines):
    if line.lstrip().__contains__("Ausdruck: Gesamte Akte"):
        headers.append(i)
    if line.lstrip().__contains__("Bei aktuell laufenden Statusmodulen"):
        skip.add(i)

for j, h in enumerate(headers):
    # Header-Zeile + die nächsten 7 Zeilen überspringen
    skip.update(range(h, min(h + 8, len(lines))))
    # Zusätzlich: die Footer-Zeile davor (außer beim allerersten Header)
    if j > 0 and h - 1 >= 0:
        skip.add(h - 1)

out_path = "data/gesamte_akte_cleared.csv"
with open(out_path, "w", encoding="utf-8") as f:
    for i, line in enumerate(lines):
        if i in skip:
            continue
        f.write(line + "\n")

In [ ]:
counter = {}
for row in raw_matrix:
    if row and row[0] is not None:
        counter[row[0]] = counter.get(row[0], 0) + 1
uniq_dict = {c: n for c, n in counter.items() if n == 1}
# uniq_dict

# Datenextraktion

In [ ]:
CSV_CLEAR = "data/gesamte_akte_cleared.csv"
file_clear = open(CSV_CLEAR, "r", encoding="utf-8").read()
lines_clear = file_clear.splitlines()
matrix = [[(c if (c:=cell.strip()) != "" else None) for cell in line.split(";")]
              for line in file_clear.splitlines()]

In [ ]:
from services.headers import headers
# headers

header_list = [x for v in headers.values() for x in v]
# header_list

In [ ]:
import re
def check_row(row):
    row_type = None
    date_re = re.compile(r"\d{2}\.\d{2}\.\d{2} \d{2}:\d{2}")

    if row[0] in header_list:
        return "header"
    if 

In [ ]:
# sammelt Zeilenblöcke nach jedem Header k mit gleicher Zeilenlänge bis zum nächsten Header
dfs = {k: [] for k in header_list}

for k in header_list:
    line = None
    # Header-Zeile finden
    for i, l in enumerate(matrix):
        if l and len(l) > 0 and l[0] == k:
            line = i
            break
    # nichts gefunden
    if line is None:
        continue

    header_line_len = len(matrix[line])  # Länge der Header-Zeile als Referenz (oder nimm matrix[line+1] falls gewünscht)

    # nachfolgende Zeilen einsammeln
    while True:
        line += 1
        if line >= len(matrix):
            break
        row = matrix[line]

        # Abbruch wenn nächste Header-Zeile beginnt
        if row and len(row) > 0 and isinstance(row[0], str) and (row[0] in header_list):
            break

        # Abbruch wenn sich die Spaltenanzahl ändert
        if len(row) != header_line_len:
            ##################### Hier weiter #####################
            ##################### Hier weiter #####################
            ##################### Hier weiter #####################
            ##################### Hier weiter #####################
            # Hier prüfen, ob eine neue Datumszeile mit unterschiedlicher Anzahl an Spalten begonnen wurde
            print("len error")
            break

        dfs[k].append(row)

In [ ]:
# das sagt chat-GPT dazu
import pandas as pd
import re

# deine Header-Definition
headers = {
    "Vitaldaten": ['Online erfasste Vitaldaten', 'Manuell erfasste Vitaldaten'],
    "Respiratordaten": ['Online erfasste Respiratorwerte', 'Beatmung', 'Manuell erfasste Respiratorwerte'],
    "Labor": [
        'Labor: Blutgase arteriell','Labor: Blutgase venös','Labor: Blutgase gv','Labor: Blutgase unspez.',
        'Labor: Blutbild','Labor: Differentialblutbild','Labor: Blutgruppe','Labor: Gerinnung',
        'Labor: TEG','Labor: TAT','Labor: Enzyme','Labor: Retention','Labor: Lipide','Labor: Proteine',
        'Labor: Elektrolyte','Labor: Blutzucker','Labor: Klinische Chemie','Labor: Medikamentenspiegel',
        'Labor: Schilddrüse','Labor: Serologie/Infektion'],
    "Dokumentation": ["Arztnotizen", "Pflegenotizen"],
    "Medikamentengaben": ["Medikamentengabe"],
    "Katheter": ["Katheter"],
    "Drainagen": ["Drainagen"],
    "Wunden": ["Wunden"],
    "ALLE Patientendaten": ["ALLE Patientendaten"]
}

# Regex für Datum
date_re = re.compile(r"\d{2}\.\d{2}\.\d{2} \d{2}:\d{2}")

# CSV einlesen (roh, ohne Header)
import pandas as pd

raw_df = pd.read_csv(
    "data/gesamte_akte_cleared.csv",
    sep=";",          # oder sep="|" testen, je nach Export
    engine="python",  # langsamer, aber robuster
    on_bad_lines="warn",  # oder "skip"
)
matrix = raw_df.fillna("").values.tolist()

records = []
current_block = None
timestamps = []

for row in matrix:
    row = [str(c).strip() for c in row]

    # 1) Blockanfang?
    if any(row[0] in vals for vals in headers.values()):
        # Bestimme Blocktyp
        for k, vals in headers.items():
            if row[0] in vals:
                current_block = k
                timestamps = []
                break
        continue

    # 2) Headerzeile (enthält Zeitstempel)?
    if sum(bool(date_re.match(c)) for c in row if c) > 2:
        timestamps = [pd.to_datetime(c, format="%d.%m.%y %H:%M") for c in row if date_re.match(c)]
        continue

    # 3) Datenzeile innerhalb eines Blocks
    if current_block and timestamps and row[2]:
        param = row[2]
        values = [c for c in row if c not in ("", None) and not date_re.match(str(c))]
        # Mapping timestamp <-> values
        for t, v in zip(timestamps, values[-len(timestamps):]):
            records.append({
                "block": current_block,
                "parameter": param,
                "timestamp": t,
                "value": v
            })

# Ergebnis-DF
df = pd.DataFrame(records)
print(df.head())

In [ ]:
print(df[10000:])

In [ ]:
test_string = datastring.splitlines()
test_lists = [[(c if (c:=cell.strip()) != "" else None) for cell in line.split(";")]
              for line in datastring.splitlines()]

[len(l) for l in test_lists]

In [ ]:
df[df["Kategorie"] == "Labor: Schilddrüse"]

In [ ]:
# Ich finde den Ansatz gar nicht schlecht, die einzelnen Daten wie von chatGPT vorgeschlagen als Blöcke/Parameter/Datum/Value zu speichern.
## Dafür muss ich eigentlich nur erst die Blöcke erkennen, dann die Zeilen mit dem Datum erkennen, das in Datetime parsen & die untergeordneten parameter einpassen